Model Comparison
================

Common questions when fitting a model to data are: what model should I use? How many parameters should the model have?
Is the model too complex or too simple?

Model comparison provides answers to these questions. It amounts to composing and fitting many different models to the data
and comparing how well they fit.

This example illustrates model comparison using the noisy 1D signal example. We fit a dataset consisting of two
Gaussians and fit it with three models comprised of 1, 2 and 3 Gaussian's respectively.

Using the Bayesian evidence to compare the models, we favour the model with 2 Gaussians, which is the "correct" model
given that it was the model used to simulate the dataset in the first place.

__Metrics__

Different metrics can be used compare models and quantify their `goodness-of-fit`.

In this example we show the results of using two different metrics:

 - `log_likelihood`: The value returned by the `log_likelihood_function` of an `Analysis` object. which is directly
   related to the sum of the residuals squared (e.g. the `chi_squared`). The log likelihood does not change when more
   or less parameters are included in the model, therefore it does not account for over-fitting and will often favour
   more complex models irrespective of whether they fit the data better.

 - `log_evidence`: The Bayesian evidence, which is closely related to the log likelihood but utilizes additional
   information which penalizes models based on their complexity. The Bayesian evidence will therefore favour simpler
   models over more complex models, unless the more complex model provides a much better fit to the data.

__Example Source Code (`af.ex`)__

The **PyAutoFit** source code has the following example objects (accessed via `af.ex`) used in this tutorial:

 - `Analysis`: an analysis object which fits noisy 1D datasets, including `log_likelihood_function` and
 `visualize` functions.

 - `Gaussian`: a model component representing a 1D Gaussian profile.

These are functionally identical to the `Analysis` and `Gaussian` objects you have seen previously.

In [ ]:
%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

import matplotlib.pyplot as plt
import numpy as np
from os import path

import autofit as af

__Data__

Load data of two 1D Gaussians from a .json file in the directory `autofit_workspace/dataset/gaussian_x2`.

This 1D data was created using two 1D Gaussians, therefore model comparison should favor a model with two Gaussians over 
a models with 1 or 3 Gaussians.

In [ ]:
dataset_path = path.join("dataset", "example_1d", "gaussian_x2")
data = af.util.numpy_array_from_json(file_path=path.join(dataset_path, "data.json"))
noise_map = af.util.numpy_array_from_json(
    file_path=path.join(dataset_path, "noise_map.json")
)

Plot the data. 

In [ ]:
xvalues = range(data.shape[0])

plt.errorbar(
    x=xvalues, y=data, yerr=noise_map, color="k", ecolor="k", linestyle=" ", elinewidth=1, capsize=2
)
plt.title("1D Gaussian Dataset Used For Model Comparison.")
plt.xlabel("x values of profile")
plt.ylabel("Profile normalization")
plt.show()
plt.close()

__Model x1 Gaussian__

Create a model to fit the data, starting with a model where the data is fitted with 1 Gaussian.

In [ ]:
model = af.Collection(gaussian_0=af.ex.Gaussian)

The `info` attribute shows the model in a readable format, showing it contains one `Gaussian`.

In [ ]:
print(model.info)

Create the analysis which fits the model to the data.

In [ ]:
analysis = af.ex.Analysis(data=data, noise_map=noise_map)

Fit the data using a non-linear search, to determine the goodness of fit of this model.

We use the nested sampling algorithm Dynesty, noting that the Bayesian evidence (`log_evidence`) of a model can only
be estimated using a nested sampling algorithm.

In [ ]:
search = af.DynestyStatic(
    path_prefix=path.join("session_3"),
    name="gaussian_x1",
    nlive=50,
    sample="rwalk",
    iterations_per_update=3000
)

Perform the fit.

In [ ]:
result_x1_gaussian = search.fit(model=model, analysis=analysis)

The results are concisely summarised using the `result.info` property.

These show that the parameters of the Gaussian are well constrained, with small errors on their inferred values.
However, it does not inform us of whether the model provides a good fit to the data overall.

In [ ]:
print(result_x1_gaussian.info)

The maximum log likelihood model is used to visualize the fit.

For 1 Gaussian, residuals are visible, whereby the model Gaussian cannot fit the highest central data-point and 
there is a mismatch at the edges of the profile around pixels 40 and 60.

Based on visual inspection, the model therefore provides a poor fit to the data.

In [ ]:
instance = result_x1_gaussian.max_log_likelihood_instance

gaussian_0 = instance.gaussian_0.model_data_1d_via_xvalues_from(
    xvalues=np.arange(data.shape[0])
)
model_data = gaussian_0

plt.errorbar(
    x=xvalues, y=data, yerr=noise_map, color="k", ecolor="k", linestyle=" ", elinewidth=1, capsize=2
)
plt.plot(range(data.shape[0]), model_data, color="r")
plt.plot(range(data.shape[0]), gaussian_0, "--")
plt.title("Model fit using 1 Gaussian.")
plt.xlabel("x values of profile")
plt.ylabel("Profile normalization")
plt.show()
plt.close()

Print the `log_likelihood` and `log_evidence` of this model-fit, which we will compare to more complex models in order 
to determine which model provides the best fit to the data.

In [ ]:
print("1 Gaussian:")
print(f"Log Likelihood: {max(result_x1_gaussian.samples.log_likelihood_list)}")
print(f"Log Evidence: {result_x1_gaussian.samples.log_evidence}")

__Model x2 Gaussian__

We now create a model to fit the data which consists of 2 Gaussians.

In [ ]:
model = af.Collection(gaussian_0=af.ex.Gaussian, gaussian_1=af.ex.Gaussian)

The `info` attribute shows the model now consists of two `Gaussian`'s.

In [ ]:
print(model.info)

We repeat the steps above to create the non-linear search and perform the model-fit.

In [ ]:
search = af.DynestyStatic(
    path_prefix=path.join("session_3"),
    name="gaussian_x2",
    nlive=50,
    sample="rwalk",
    iterations_per_update=3000
)

result_x2_gaussian = search.fit(model=model, analysis=analysis)

The results show that two Gaussians have now been fitted to the data.

In [ ]:
print(result_x2_gaussian.info)

Visualizing the fit, we see that the problems with the previous fit have been addressed. The central data-point at the 
highest normalization is fitted correctly and the residuals at the edges of the profile around pixels 40 and 60 are 
significantly reduced.

There are effectively no residuals, indicating that the model provides a good fit to the data.

The residuals are so small that they are consistent with noise in the data. One therefore should not expect that 
a more complex model than one with 2 Gaussians can provide a better fit.

In [ ]:
instance = result_x2_gaussian.max_log_likelihood_instance

gaussian_0 = instance.gaussian_0.model_data_1d_via_xvalues_from(
    xvalues=np.arange(data.shape[0])
)
gaussian_1 = instance.gaussian_1.model_data_1d_via_xvalues_from(
    xvalues=np.arange(data.shape[0])
)
model_data = gaussian_0 + gaussian_1

plt.errorbar(
    x=xvalues, y=data, yerr=noise_map, color="k", ecolor="k", linestyle=" ", elinewidth=1, capsize=2
)
plt.plot(range(data.shape[0]), model_data, color="r")
plt.plot(range(data.shape[0]), gaussian_0, "--")
plt.plot(range(data.shape[0]), gaussian_1, "--")
plt.title("Model fit using 2 Gaussian.")
plt.xlabel("x values of profile")
plt.ylabel("Profile normalization")
plt.show()
plt.close()

Print the `log_likelihood` and `log_evidence` of this model-fit, and compare these values to the previous model-fit
which used 1 Gaussian.

In [ ]:
print("1 Gaussian:")
print(f"Log Likelihood: {max(result_x1_gaussian.samples.log_likelihood_list)}")
print(f"Log Evidence: {result_x1_gaussian.samples.log_evidence}")

print("2 Gaussians:")
print(f"Log Likelihood: {max(result_x2_gaussian.samples.log_likelihood_list)}")
print(f"Log Evidence: {result_x2_gaussian.samples.log_evidence}")

Both the `log_likelihood` and `log_evidence` have increased significantly, indicating that the model with 2 Gaussians
is favored over the model with 1 Gaussian.

This is expected, as we know the data was generated using 2 Gaussians!

__Model x3 Gaussian__

We now create a model to fit the data which consists of 3 Gaussians.

In [ ]:
model = af.Collection(gaussian_0=af.ex.Gaussian, gaussian_1=af.ex.Gaussian, gaussian_2=af.ex.Gaussian)

The `info` attribute shows the model consists of three `Gaussian`'s.

In [ ]:
print(model.info)

We repeat the steps above to create the non-linear search and perform the model-fit.

In [ ]:
search = af.DynestyStatic(
    path_prefix=path.join("session_3"),
    name="gaussian_x3",
    nlive=50,
    sample="rwalk",
    iterations_per_update=3000
)

result_x3_gaussian = search.fit(model=model, analysis=analysis)

The results show that three Gaussians have now been fitted to the data.

In [ ]:
print(result_x3_gaussian.info)

Visualizing the fit, we see that there are effectively no residuals, indicating that the model provides a good fit.

By eye, this fit looks as good as the 2 Gaussian model above.

In [ ]:
instance = result_x3_gaussian.max_log_likelihood_instance

gaussian_0 = instance.gaussian_0.model_data_1d_via_xvalues_from(
    xvalues=np.arange(data.shape[0])
)
gaussian_1 = instance.gaussian_1.model_data_1d_via_xvalues_from(
    xvalues=np.arange(data.shape[0])
)
gaussian_2 = instance.gaussian_2.model_data_1d_via_xvalues_from(
    xvalues=np.arange(data.shape[0])
)
model_data = gaussian_0 + gaussian_1 + gaussian_2

plt.errorbar(
    x=xvalues, y=data, yerr=noise_map, color="k", ecolor="k", linestyle=" ", elinewidth=1, capsize=2
)
plt.plot(range(data.shape[0]), model_data, color="r")
plt.plot(range(data.shape[0]), gaussian_0, "--")
plt.plot(range(data.shape[0]), gaussian_1, "--")
plt.plot(range(data.shape[0]), gaussian_2, "--")
plt.title("Model fit using 3 Gaussian.")
plt.xlabel("x values of profile")
plt.ylabel("Profile normalization")
plt.show()
plt.close()

We print the `log_likelihood` and `log_evidence` of this model-fit, and compare these values to the previous model-fit
which used 1 and 2 Gaussian.

In [ ]:
print("1 Gaussian:")
print(f"Log Likelihood: {max(result_x1_gaussian.samples.log_likelihood_list)}")
print(f"Log Evidence: {result_x1_gaussian.samples.log_evidence}")

print("2 Gaussians:")
print(f"Log Likelihood: {max(result_x2_gaussian.samples.log_likelihood_list)}")
print(f"Log Evidence: {result_x2_gaussian.samples.log_evidence}")

print("3 Gaussians:")
print(f"Log Likelihood: {max(result_x3_gaussian.samples.log_likelihood_list)}")
print(f"Log Evidence: {result_x3_gaussian.samples.log_evidence}")

We now see an interesting result. The `log_likelihood` of the 3 Gaussian model is higher than the 2 Gaussian model
(albeit, only slightly higher). However, the `log_evidence` is lower than the 2 Gaussian model.

This confirms the behavior discussed at the start of the tutorial. The Bayesian evidence penalizes models with more 
freedom to fit the data, unless they provide a significantly better fit to the data. Using the evidence we favor the
2 Gaussian model over the 3 Gaussian model for this reason, whereas using the likelihood we favor the 3 Gaussian model.

__Wrap Up__

We have shown how one can perform Bayesian model comparison, in order to determine which model provides the best-fit
to the data. We saw how the Bayesian evidence penalizes models with more freedom to fit the data, unless they provide
a significantly better fit to the data.

For your research problem, now consider if there are multiple models that you could use to fit your data. Do you know
which will provide the best-fit? Or will you need to use model comparison to determine this? Will certain models work
on datasets of a certain quality, but simpler models be required for lower quality datasets?

Multiple Datasets
=================

It is common to have multiple observations of the same signal. For the 1D Gaussian example, this would be multiple
1D datasets of the same underlying Gaussian, with different noise-map realizations. In this situation, fitting the
same model to all datasets simultaneously is desired, and would provide better constraints on the model.

On other occations, the signal may vary across the datasets in a way that requires that the model is updated
accordingly. For example, a scenario where the centre of each Gaussian is the same across the datasets, but
their `sigma` values are different in each dataset. A model where all Gaussians share the same `centre` is now required.

This examples illustrates how to perform model-fits to multiple datasets simultaneously, including tools to customize
the model composition such that specific parameters of the model vary across the datasets.

This uses the summing of `Analysis` object, which each have their own unique dataset and `log_likelihood_function`.
Unique `Analysis` objects can be written for each dataset, meaning that we can perform model-fits to diverse datasets
with different formats and structures.

It is also common for each individual dataset to only constrain specific aspects of a model. The high level of model
customizaiton ensures that composing a model that is appropriate for fitting to such large datasets is straight
forward.

__Example Source Code (`af.ex`)__

The **PyAutoFit** source code has the following example objects (accessed via `af.ex`) used in this tutorial:

 - `Analysis`: an analysis object which fits noisy 1D datasets, including `log_likelihood_function` and 
 `visualize` functions.

 - `Gaussian`: a model component representing a 1D Gaussian profile.

These are functionally identical to the `Analysis` and `Gaussian` objects you have seen elsewhere in the workspace.

In [ ]:
%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

import matplotlib.pyplot as plt
import numpy as np
from os import path

import autofit as af
import autofit.plot as aplt

__Data__

First, lets load 3 datasets of a 1D Gaussian, by loading them from .json files in the directory 
`autofit_workspace/dataset/`.

All three datasets contain an identical signal, meaning that it is appropriate to fit the same model to all three 
datasets simultaneously.

Each dataset has a different noise realization, meaning that performing a simultaneously fit will offer improved 
constraints over individual fits.

In [ ]:
dataset_size = 3

data_list = []
noise_map_list = []

for dataset_index in range(dataset_size):
    dataset_path = path.join(
        "dataset", "example_1d", f"gaussian_x1_identical_{dataset_index}"
    )

    data = af.util.numpy_array_from_json(file_path=path.join(dataset_path, "data.json"))
    data_list.append(data)

    noise_map = af.util.numpy_array_from_json(
        file_path=path.join(dataset_path, "noise_map.json")
    )
    noise_map_list.append(noise_map)

Now lets plot all 3 datasets, including their error bars. 

In [ ]:
for data, noise_map in zip(data_list, noise_map_list):
    xvalues = range(data.shape[0])

    plt.errorbar(
        x=xvalues,
        y=data,
        yerr=noise_map,
        color="k",
        ecolor="k",
        linestyle=" ",
        elinewidth=1,
        capsize=2,
    )
    plt.show()
    plt.close()

__Model__

Next, we create our model, which corresponds to a single 1D Gaussian, that is fitted to all 3 datasets simultaneously.

In [ ]:
model = af.Model(af.ex.Gaussian)

Checkout `autofit_workspace/config/priors/model.yaml`, this config file defines the default priors of the `Gaussian` 
model component. 

We overwrite the priors below to make them explicit.

In [ ]:
model.centre = af.UniformPrior(lower_limit=0.0, upper_limit=100.0)
model.normalization = af.LogUniformPrior(lower_limit=1e-2, upper_limit=1e2)
model.sigma = af.GaussianPrior(
    mean=10.0, sigma=5.0, lower_limit=0.0, upper_limit=np.inf
)

__Analysis__

We set up our three instances of the `Analysis` class, which is the same as the one we wrote in a previous session.

We set up an `Analysis` for each dataset one-by-one, using a for loop:

In [ ]:
analysis_list = []

for data, noise_map in zip(data_list, noise_map_list):
    analysis = af.ex.Analysis(data=data, noise_map=noise_map)
    analysis_list.append(analysis)

__Analysis Summing__

We now sum together every analysis in the list, to produce an overall analysis class which we fit with the non-linear
search.

By summing analysis objects the following happen:

 - The log likelihood values computed by the `log_likelihood_function` of each individual analysis class are summed to
   give an overall log likelihood value that the non-linear search uses for model-fitting.

 - The output path structure of the results goes to a single folder, which includes sub-folders for the visualization
   of every individual analysis object based on the `Analysis` object's `visualize` method.

In [ ]:
analysis = analysis_list[0] + analysis_list[1] + analysis_list[2]

We can alternatively sum a list of analysis objects as follows:

In [ ]:
analysis = sum(analysis_list)

The `log_likelihood_function`'s can be called in parallel over multiple cores by changing the `n_cores` parameter:

In [ ]:
analysis.n_cores = 1

__Search__

To fit multiple datasets via a non-linear search we use this summed analysis object:

In [ ]:
search = af.DynestyStatic(
    path_prefix=path.join("session_3"),
    name="multiple_datasets_simple",
    sample="rwalk",
    iterations_per_update=3000
)

result_list = search.fit(model=model, analysis=analysis)

__Result__

The result object returned by the fit is a list of the `Result` objects you have used in other examples.

In this example, the same model is fitted across all analyses, thus every `Result` in the `result_list` contains
the same information on the samples and thus gives the same output from methods such 
as `max_log_likelihood_instance`.

In [ ]:
print(result_list[0].max_log_likelihood_instance.sigma)
print(result_list[1].max_log_likelihood_instance.sigma)

We can plot the model-fit to each dataset by iterating over the results:

In [ ]:
for data, result in zip(data_list, result_list):
    instance = result.max_log_likelihood_instance

    model_data = instance.model_data_1d_via_xvalues_from(
        xvalues=np.arange(data.shape[0])
    )

    plt.errorbar(
        x=xvalues,
        y=data,
        yerr=noise_map,
        color="k",
        ecolor="k",
        elinewidth=1,
        capsize=2,
    )
    plt.plot(xvalues, model_data, color="r")
    plt.title("Dynesty model fit to 1D Gaussian dataset.")
    plt.xlabel("x values of profile")
    plt.ylabel("Profile normalization")
    plt.show()
    plt.close()

__Variable Model Across Datasets__

Above, the same model was fitted to every dataset simultaneously, which was possible because all 3 datasets contained 
an identical signal with only the noise varying across the datasets.

It is common for the signal in each dataset to be different and for it to constrain only certain aspects of the model.
The model parameterization therefore needs to change in order to account for this.

Lets look at an example of a dataset of 3 1D Gaussians where the signal varies across the datasets:

In [ ]:
dataset_path = path.join("dataset", "example_1d", "gaussian_x1_variable")

dataset_name_list = ["sigma_0", "sigma_1", "sigma_2"]

data_list = []
noise_map_list = []

for dataset_name in dataset_name_list:
    dataset_time_path = path.join(dataset_path, dataset_name)

    data = af.util.numpy_array_from_json(
        file_path=path.join(dataset_time_path, "data.json")
    )
    noise_map = af.util.numpy_array_from_json(
        file_path=path.join(dataset_time_path, "noise_map.json")
    )

    data_list.append(data)
    noise_map_list.append(noise_map)

If we plot these datasets, we see that the `sigma` of each Gaussian decreases.

We will illustrate models which vary over the data based on this `sigma` value. 

In [ ]:
for data, noise_map in zip(data_list, noise_map_list):
    xvalues = range(data.shape[0])

    af.ex.plot_profile_1d(xvalues=xvalues, profile_1d=data)

In this case, the `centre` and `normalization` of all three 1D Gaussians are the same in each dataset,
but their `sigma` values are decreasing.

We therefore wish to compose and to fit a model to all three datasets simultaneously, where the `centre` 
and `normalization` are the same across all three datasets but the `sigma` value is unique for each dataset.

To do that, we interface a model with a summed list of analysis objects, which we create below for this new
dataset:

In [ ]:
analysis_list = []

for data, noise_map in zip(data_list, noise_map_list):
    analysis = af.ex.Analysis(data=data, noise_map=noise_map)
    analysis_list.append(analysis)

analysis = sum(analysis_list)

We next compose a model of a 1D Gaussian, as performed frequently throughout the **PyAutoFit** examples:

In [ ]:
model = af.Collection(gaussian=af.Model(af.ex.Gaussian))

We now update the model using the summed `Analysis `objects to compose a model where: 

 - The `centre` and `normalization` values of the Gaussian fitted to every dataset in every `Analysis` object are
 identical. 

 - The `sigma` value of the every Gaussian fitted to every dataset in every `Analysis` object are different.

This means that the model has 5 free parameters in total, the shared `centre` and `normalization` and a unique
`sigma` value for every dataset.

In [ ]:
analysis = analysis.with_free_parameters(model.gaussian.sigma)

To inspect this new model, with extra parameters for each dataset created, we have to extract the modified
version of this model from the `Analysis` object.

This occurs automatically when we begin a non-linear search, therefore the normal `model` we created above
is what we input to the `search.fit()` method.

In [ ]:
model_updated = analysis.modify_model(model)

This means that the model has 5 free parameters in total, the shared `centre` and `normalization` and a unique
`sigma` value for every dataset.

In [ ]:
print(model_updated.total_free_parameters)

We can now fit this model to the data using the usual **PyAutoFit** tools:

In [ ]:
search = af.DynestyStatic(
    path_prefix=path.join("session_3"),
    name="multiple_datasets_free_sigma",
    sample="rwalk"
)

result_list = search.fit(model=model, analysis=analysis)

__Variable Parameters As Relationship__

In the model above, an extra free parameter `sigma` was added for every dataset. 

This was ok for the simple model fitted here, to just 3 datasets. However, for more complex models and problems
with 10+ datasets one will quickly find that the model complexity increases dramatically.

In these circumstances, one can instead compose a model where the parameters vary smoothly across the datasets
via a user defined relation.

Below, we compose a model where the `sigma` value fitted to each dataset is computed according to:

 `y = m * x + c` : `sigma` = sigma_m * x + sigma_c`

Where x is an integer number specifying the index of the dataset (e.g. 1, 2 and 3).

By defining a relation of this form, `sigma_m` and `sigma_c` are the only free parameters of the model which vary
across the datasets. 

Therefore, if more datasets are added the number of model parameter does not increase, like we saw above.

In [ ]:
sigma_m = af.UniformPrior(lower_limit=-10.0, upper_limit=10.0)
sigma_c = af.UniformPrior(lower_limit=-10.0, upper_limit=10.0)

x_list = [1.0, 2.0, 3.0]

analysis_with_relation_list = []

for x, analysis in zip(x_list, analysis_list):
    sigma_relation = (sigma_m * x) + sigma_c

    analysis_with_relation = analysis.with_model(
        model.replacing({model.gaussian.sigma: sigma_relation})
    )

    analysis_with_relation_list.append(analysis_with_relation)

We can fit this model as per usual, you may wish to checkout the `model.info` file to see how a schematic of this
model's composition.

In [ ]:
analysis_with_relation = sum(analysis_with_relation_list)

search = af.DynestyStatic(
    path_prefix=path.join("session_3"),
    name="multiple_datasets_relation",
    sample="rwalk"
)

result_list = search.fit(model=model, analysis=analysis_with_relation)

__Temporally Varying Models__

An obvious example of fitting models which vary across datasets are time-varying models, where the datasets are
observations of a signal which varies across time.

In such circumstances, it is common for certain model parameters to be known to not vary as a function of time (and 
therefore be fixed across the datasets) whereas other parameters are known to vary as a function of time (and therefore
should be parameterized accordingly using the API illustrated here).

__Different Analysis Objects__

For simplicity, this example summed together only a single `Analysis` class. 

For many problems one may have multiple datasets which are quite different in their format and structure (perhaps 
one is a  1D signal whereas another dataset is an image). In this situation, one can simply define unique `Analysis`
objects for each type of dataset, which will contain a unique `log_likelihood_function` and methods for visualization.

Nevertheless, the analysis summing API illustrated here will still work, meaning that **PyAutoFit** makes it simple to 
fit highly customized models to multiple datasets that are different in their format and structure. 

__Wrap Up__

We have shown how to fit large datasets simultaneously, using custom models that vary specific
parameters across the dataset.

Now think about your specific research problem, and whether you may find yourself in a situation where you wish to
fit a model to multiple datasets simultaneously. If so, how might the model need to be composed to achieve this?
Will parameters be shared across datasets, or will they vary? Will the datasets be of the same format and structure,
or different?